This is necessary for having figures directly in the notebook.

In [1]:
%matplotlib inline

Import necessary modules

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import seaborn as sns

from tridesclous import SpikeSorter, CatalogueWindow

We download Locust dataset from zenedo.
We take 3 trials in this file.

In [3]:
from urllib.request import urlretrieve
import os
import h5py

name = 'locust20010201.hdf5'
distantfile = 'https://zenodo.org/record/21589/files/'+name
localfile = name
if not os.path.exists(localfile):
    urlretrieve(distantfile, localfile)
hdf = h5py.File(localfile,'r')

# read 3 trials (=3 segments)
ch_names = ['ch09','ch11','ch13','ch16']
trial_names = ['trial_01', 'trial_02', 'trial_03']
sigs_by_trials = []
for trial_name in trial_names:
    sigs = np.array([hdf['Continuous_1'][trial_name][name][...] for name in ch_names]).transpose()
    sigs = (sigs.astype('float32') - 2**15.) / 2**15
    sigs_by_trials.append(sigs)

sampling_rate = 15000.

# SpikeSorter: insert several segments

Create our data manager and append some data into it.
Note that our data is already filtered.

In [4]:
spikesorter = SpikeSorter(dirname = 'Dataset several segment')

for seg_num in range(3):
    sigs = sigs_by_trials[seg_num]
    spikesorter.dataio.append_signals_from_numpy(sigs, seg_num = seg_num,
                t_start = 0.+5*seg_num, sampling_rate =  sampling_rate, 
                signal_type = 'unfiltered', channels = ch_names)

In [5]:
print(spikesorter.summary(level=1))

# Do some smooth

In [6]:
spikesorter.apply_filter(highpass_freq = 0., box_smooth = 3)

# Detect peak an dextract waveform in one function

In [7]:
spikesorter.detect_peaks_extract_waveforms(seg_nums = 'all',  threshold=-4, 
                                    peak_sign = '-', n_span = 2, n_left=-30, n_right=50)
print(spikesorter.summary(level=1))

# project and find cluster

In [8]:
spikesorter.project(method = 'pca', n_components = 5)
spikesorter.find_clusters(7)
spikesorter.refresh_colors(reset=True, palette = 'husl')
print(spikesorter.summary(level=1))
print(spikesorter.cluster_count)

# Open interactive windows

In [9]:
%gui qt4
import pyqtgraph as pg
app = pg.mkQApp()
win = CatalogueWindow(spikesorter)
win.show()

In [12]:
spikesorter.appy_peeler(seg_nums = 'all', levels = [0, 1])

In [13]:
spiketrains = spikesorter.dataio.get_spiketrains(seg_num=0)
spiketrains

In [14]:
spiketrains = spikesorter.dataio.get_spiketrains(seg_num=1)
spiketrains

In [15]:
spiketrains = spikesorter.dataio.get_spiketrains(seg_num=2)
spiketrains